In [9]:
from math import floor, ceil
from statistics import mean
from scipy.optimize import brentq
import numpy as np

In [10]:
race_length = 9000  # s
long_stop_time = 180  # s
long_stops = 1

In [11]:
# GT3
base_pitstop_loss = 24
# TODO: Separate losses for in, outlap? Would be messy because of finish line location varying between tracks, along with fact that some pit boxes are before finish line, some after
fuel_tank_size = 100
refuel_rate = 0.353  # sec / L
tire_swap_time = 30

In [4]:
# LMP2
base_pitstop_loss = 28  # Temp value from gt3!
fuel_tank_size = 75
refuel_rate = 0.6
tire_swap_time = 10

In [4]:
def estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint = None):  # TODO multi-driver support, optional parameter for time remaining in the race
    global race_length
    global long_stops
    global long_stop_time
    global base_pitstop_loss

    global fuel_tank_size
    global refuel_rate
    global tire_swap_time

    total_long_stop_time = long_stops * long_stop_time
    effective_race_length = race_length - total_long_stop_time

    if laps_per_stint:
        liters_to_refuel = laps_per_stint * average_fuel_consumption
    else:
        liters_to_refuel = fuel_tank_size - average_fuel_consumption  # To get a lower bound on max_stint_length, we lower bound pitstop length by
        # assuming there is at least one lap of fuel left in the tank when coming in

    pitstop_length = base_pitstop_loss + max(tire_swap_time, refuel_rate * liters_to_refuel)

    if not laps_per_stint:
        laps_per_stint = floor(fuel_tank_size / average_fuel_consumption)  # TODO Can this be moved up to avoid having the previous if laps_per_stint block?
    
    max_stint_length = average_lap_time * laps_per_stint + pitstop_length
    # TODO: Parameter for max laps out? (tire strategy, may be better to save for new strat calculator)

    stint_total = (effective_race_length + average_lap_time) / max_stint_length  # Adding average_lap_time to numerator accounts for 6 hours + 1 lap
    # We want an upper bound of this
    # PROBLEM! Does not account for the fact that long stops have free pit stop!!!  Long stops are removed from numerator but also need to be removed from denominator
    # To fix, max_stint_length should be a weighted average of max_stint_length
    average_stint_length = average_lap_time * laps_per_stint + ((stint_total - long_stops) / stint_total) * pitstop_length
    new_stint_total = (effective_race_length + average_lap_time) / average_stint_length  # TODO make this a loop that breaks once it converges well enough

    # pitstop_length is the sum of the time lost between the inlap and outlap. The first stint does not have an outlap but rather a formation lap (which is longer than an outlap),
    # the final stint has an outlap but not an inlap. Therefore right now stint_total is overestimated by outlap_differential / max_stint_length

    seconds_margin = (round(new_stint_total) - new_stint_total) * average_stint_length
    
    return new_stint_total, max_stint_length, seconds_margin  # Separate function for max_stint_length?



In [5]:
def laps_and_fuel_per_stint(average_lap_time, average_fuel_consumption, num_pitstops):  # TODO: Consider the fact that long stops should refuel more than short stops
    global fuel_tank_size
    stint_total = num_pitstops + 1
    obj = lambda laps_per_stint: estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint)[0] - stint_total

    # Calculate optimal laps per stint
    optimal_laps_per_stint = brentq(obj, 1, floor(fuel_tank_size / average_fuel_consumption))

    # Return liters required to do optimal laps
    return floor(optimal_laps_per_stint), ceil(average_fuel_consumption * optimal_laps_per_stint)

In [6]:
def pit_time_matrix(average_lap_time, average_fuel_consumption, num_pitstops):  # Number of completed stints on axis 0, number of completed long stops on axis 1
    # TODO: optional argument for start time
    global long_stops
    global long_stop_time
    global race_length
    global base_pitstop_loss
    global tire_swap_time
    global refuel_rate

    laps_per_stint, liters_to_refuel = laps_and_fuel_per_stint(average_lap_time, average_fuel_consumption, num_pitstops)
    _, stint_length, _ = estimate_stint_length(average_lap_time, average_fuel_consumption, laps_per_stint)
    pitstop_length = base_pitstop_loss + max(tire_swap_time, refuel_rate * liters_to_refuel)

    # Create 1D arrays
    long_stop_array = np.arange(long_stops + 1).reshape((1, -1))
    pit_stops_array_base = np.arange(1, num_pitstops + 1).reshape((-1, 1))  # First element is before first pitstop (after first stint), last element is final pitstop

    # Convert pit_stops_array to times that car should enter pits if there are no long stops
    pit_stops_array = race_length - (pit_stops_array_base * stint_length) + pitstop_length  # Adding back pitstop_length is necessary since the pitstop hasn't been done on entry
    pit_stops_array = pit_stops_array.astype(np.int32)

    # Factor in long stops
    pit_stops_matrix = (pit_stops_array - long_stop_time * long_stop_array)

    # Convert array to timedelta
    pit_stops_matrix_seconds = (pit_stops_matrix % 60).astype(np.str_)
    pit_stops_matrix_minutes = ((pit_stops_matrix // 60) % 60).astype(np.str_)
    pit_stops_matrix_hours = (pit_stops_matrix // 3600).astype(np.str_)
    pit_stops_matrix_string = pit_stops_matrix_hours + ":" + pit_stops_matrix_minutes + ":" + pit_stops_matrix_seconds  # Consider printing strings line by line

    # Replace invalid elements of array with NaN (fewer long stops than stops completed and more long stops than pit stops remaining)
    pit_stops_matrix_string[(pit_stops_array_base <= long_stop_array) | (pit_stops_array_base - 1 > (len(long_stop_array) + long_stop_array))] = "NaN"

    return pit_stops_matrix_string

In [32]:
pit_time_matrix(91.7, 1.7, 3)  # lmp2

array([['3:1:40', 'NaN', 'NaN'],
       ['2:2:13', '1:59:13', 'NaN'],
       ['NaN', '0:59:46', '0:56:46']], dtype='<U65')

In [58]:
pit_time_matrix((103.458 + 104) / 2, (2.9 + 2.93) / 2, 3)  # Rake/Marcus hybrid

array([['3:2:57', 'NaN', 'NaN'],
       ['2:4:55', '2:1:55', 'NaN'],
       ['NaN', '1:3:53', '1:0:53']], dtype='<U65')

In [12]:
laps_and_fuel_per_stint(103.557, 2.96, 2)  # Rake

(28, 84)

In [7]:
estimate_stint_length(103.557, 2.96)  # Rake

(2.5843312165886796, 3475.6361199999997, 1435.2820018123602)

In [ ]:
estimate_stint_length(92.2, 1.72)  # Marv

(2.2258398443459813, 4036.5679999999998, -904.2563713166232)

In [10]:
laps_and_fuel_per_stint(92.2, 1.72, 2)

(31, 55)

In [4]:
car_1 = [(93, 2.55), (92, 2.7), (90.5, 2.65)]  # Rake, Dadeler
car_2 = [(92, 2.7), (93.4, 2.61), (91.7, 2.6)]  # Dadeler, Marcus, Marv
car_2_mod = car_2[1:]

print(f"Car 1 stints: {mean([estimate_stint_length(*x)[0] for x in car_1])}")
print(f"Car 2 stints: {mean([estimate_stint_length(*x)[0] for x in car_2])}")
print(f"Car 2 stints without dadeler: {mean([estimate_stint_length(*x)[0] for x in car_2_mod])}")

Car 1 stints: 6.008526530400015
Car 2 stints: 5.970472025708266
Car 2 stints without dadeler: 5.90643506426124
